# **Лабораторная работа №4**
# **Выполнил студент группы БФИ 2302 Рыбаков А.Д.**

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)

### Задание №1

In [ ]:
import time
import random
import bisect
from collections import deque

# Генерация случайных данных
def generate_data(size, start=0, end=1000):
    return sorted(random.sample(range(start, end), size))

# Бинарный поиск (итеративный)
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1

# Узел бинарного дерева
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

# Бинарное дерево (с итеративной вставкой)
class BinaryTree:
    def __init__(self):
        self.root = None

    def insert(self, value):
        new_node = TreeNode(value)
        if not self.root:
            self.root = new_node
            return

        current = self.root
        while True:
            if value < current.value:
                if current.left is None:
                    current.left = new_node
                    break
                current = current.left
            else:
                if current.right is None:
                    current.right = new_node
                    break
                current = current.right

    def search(self, value):
        current = self.root
        while current:
            if current.value == value:
                return True
            elif value < current.value:
                current = current.left
            else:
                current = current.right
        return False

# Фибоначчиев поиск
def fibonacci_search(arr, target):
    size = len(arr)
    fib2, fib1 = 0, 1
    fibM = fib1 + fib2
    while fibM < size:
        fib2, fib1 = fib1, fibM
        fibM = fib1 + fib2

    offset = -1
    while fibM > 1:
        i = min(offset + fib2, size - 1)
        if arr[i] < target:
            fibM, fib1, fib2 = fib1, fib2, fib1 - fib2
            offset = i
        elif arr[i] > target:
            fibM, fib1, fib2 = fib2, fib1 - fib2, fib2 - fib1
        else:
            return i
    if fib1 and offset + 1 < size and arr[offset + 1] == target:
        return offset + 1
    return -1

# Интерполяционный поиск
def interpolation_search(arr, target):
    low, high = 0, len(arr) - 1
    while low <= high and arr[low] <= target <= arr[high]:
        pos = low + ((target - arr[low]) * (high - low) // (arr[high] - arr[low]))
        if arr[pos] == target:
            return pos
        if arr[pos] < target:
            low = pos + 1
        else:
            high = pos - 1
    return -1

# Функция для измерения времени работы алгоритмов
def measure_time(search_func, arr, target):
    start_time = time.time()
    result = search_func(arr, target)
    end_time = time.time()
    return result, end_time - start_time

# Генерация данных и тестирование
size = 1000
data = generate_data(size)
random.shuffle(data)  # Перемешивание данных перед вставкой в дерево
target = random.choice(data)

# Создание и наполнение бинарного дерева
tree = BinaryTree()
for num in data:
    tree.insert(num)

# Замер времени работы
print("Binary Search:", measure_time(binary_search, data, target))
print("Fibonacci Search:", measure_time(fibonacci_search, data, target))
print("Interpolation Search:", measure_time(interpolation_search, data, target))
print("Built-in bisect:", measure_time(lambda arr, x: bisect.bisect_left(arr, x), data, target))
print("Binary Tree Search:", measure_time(lambda arr, x: tree.search(x), data, target))


Binary Search: (-1, 8.58306884765625e-06)
Fibonacci Search: (-1, 1.0728836059570312e-05)
Interpolation Search: (-1, 5.4836273193359375e-06)
Built-in bisect: (440, 3.814697265625e-06)
Binary Tree Search: (True, 6.9141387939453125e-06)


### Задание №2

In [3]:
import random
from typing import List, Optional

class HashTableBase:
    def __init__(self, size: int):
        self.size = size
        self.table = [None] * size

    def _hash(self, key: str) -> int:
        # Простой хеш-функция: сумма ASCII-значений символов ключа % размер таблицы
        return sum(ord(c) for c in key) % self.size

    def insert(self, key: str, value: any) -> None:
        raise NotImplementedError()

    def search(self, key: str) -> Optional[any]:
        raise NotImplementedError()

    def delete(self, key: str) -> None:
        raise NotImplementedError()


# 1. Простое рехэширование (линейное пробирование)
class LinearProbingHashTable(HashTableBase):
    def insert(self, key: str, value: any) -> None:
        index = self._hash(key)
        while self.table[index] is not None and self.table[index][0] != key:
            index = (index + 1) % self.size  # Линейное пробирование
        self.table[index] = (key, value)

    def search(self, key: str) -> Optional[any]:
        index = self._hash(key)
        while True:
            if self.table[index] is None:
                return None  # Ключ не найден
            if self.table[index][0] == key:
                return self.table[index][1]  # Найдено значение
            index = (index + 1) % self.size  # Продолжаем поиск

    def delete(self, key: str) -> None:
        index = self._hash(key)
        while True:
            if self.table[index] is None:
                return  # Ключ не найден
            if self.table[index][0] == key:
                self.table[index] = None  # Удаляем элемент
                return
            index = (index + 1) % self.size


# 2. Рехэширование с помощью псевдослучайных чисел
class RandomProbingHashTable(HashTableBase):
    def insert(self, key: str, value: any) -> None:
        index = self._hash(key)
        step = random.randint(1, self.size - 1)  # Случайный шаг
        while self.table[index] is not None and self.table[index][0] != key:
            index = (index + step) % self.size  # Случайное пробирование
        self.table[index] = (key, value)

    def search(self, key: str) -> Optional[any]:
        index = self._hash(key)
        step = random.randint(1, self.size - 1)  # Случайный шаг
        while True:
            if self.table[index] is None:
                return None  # Ключ не найден
            if self.table[index][0] == key:
                return self.table[index][1]  # Найдено значение
            index = (index + step) % self.size  # Продолжаем поиск

    def delete(self, key: str) -> None:
        index = self._hash(key)
        step = random.randint(1, self.size - 1)  # Случайный шаг
        while True:
            if self.table[index] is None:
                return  # Ключ не найден
            if self.table[index][0] == key:
                self.table[index] = None  # Удаляем элемент
                return
            index = (index + step) % self.size


# 3. Метод цепочек
class ChainingHashTable(HashTableBase):
    def __init__(self, size: int):
        super().__init__(size)
        self.table = [[] for _ in range(size)]  # Каждая ячейка - список

    def insert(self, key: str, value: any) -> None:
        index = self._hash(key)
        for i, (k, v) in enumerate(self.table[index]):
            if k == key:
                self.table[index][i] = (key, value)  # Обновляем значение
                return
        self.table[index].append((key, value))  # Добавляем новый элемент

    def search(self, key: str) -> Optional[any]:
        index = self._hash(key)
        for k, v in self.table[index]:
            if k == key:
                return v  # Найдено значение
        return None  # Ключ не найден

    def delete(self, key: str) -> None:
        index = self._hash(key)
        for i, (k, v) in enumerate(self.table[index]):
            if k == key:
                del self.table[index][i]  # Удаляем элемент
                return


# Тестирование
if __name__ == "__main__":
    # Тестовая функция для всех методов
    def test_hash_table(table: HashTableBase):
        table.insert("apple", 10)
        table.insert("banana", 20)
        table.insert("cherry", 30)
        print(table.search("apple"))  # Should print 10
        print(table.search("banana"))  # Should print 20
        table.delete("banana")
        print(table.search("banana"))  # Should print None

    # Тестирование простого рехэширования
    print("Простое рэхеширование:")
    linear_table = LinearProbingHashTable(10)
    test_hash_table(linear_table)

    # Тестирование рехэширования с псевдослучайными числами
    print("\nПсевдослучайные числа:")
    random_table = RandomProbingHashTable(10)
    test_hash_table(random_table)

    # Тестирование метода цепочек
    print("\nМетод цепочек:")
    chaining_table = ChainingHashTable(10)
    test_hash_table(chaining_table)

Простое рэхеширование:
10
20
None

Псевдослучайные числа:
10
20
None

Метод цепочек:
10
20
None


### Задание №3

In [4]:
def print_board(board):
    for row in board:
        print(" ".join("Q" if cell else "." for cell in row))
    print("\n")

# Функция для проверки безопасности размещения ферзя в позиции (row, col)
def is_safe(board, row, col, n):
    for i in range(row):
        if board[i][col]:
            return False

    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j]:
            return False

    for i, j in zip(range(row, -1, -1), range(col, n)):
        if board[i][j]:
            return False

    return True

# Рекурсивная функция для размещения 8 ферзей
def solve_n_queens(board, row, n):
    if row == n:
        print_board(board)
        return True

    for col in range(n):
        if is_safe(board, row, col, n):
            board[row][col] = True
            if solve_n_queens(board, row + 1, n):
                return True
            board[row][col] = False

    return False

# Основная функция для запуска решения
n = 8
board = [[False] * n for _ in range(n)]
solve_n_queens(board, 0, n)


Q . . . . . . .
. . . . Q . . .
. . . . . . . Q
. . . . . Q . .
. . Q . . . . .
. . . . . . Q .
. Q . . . . . .
. . . Q . . . .




True